In [1]:
msg = "hello world"
print(msg)

hello world


In [2]:
# %pip uninstall googletrans
# %pip install googletrans==3.1.0a0
# %pip install google_trans_new

In [3]:
# %pip install pandas

In [4]:
import pandas as pd
import numpy as np
from googletrans import Translator
# from google_trans_new import google_translator
import os
import re

In [22]:
current_path = os.getcwd()

test_data_path = os.path.join(current_path, 'test_data.txt')
row_dev_posts_path = os.path.join(current_path, 'posts.txt')
row_test_posts_path = os.path.join(current_path, 'posts_groundtruth.txt')
train_ekphrasis_path = os.path.join(current_path, 'train_ekphrasis.csv')
dev_ekphrasis_path = os.path.join(current_path, 'dev_ekphrasis.csv')
test_ekphrasis_path = os.path.join(current_path, 'test_ekphrasis.csv')

processed_train_ekphrasis_path = os.path.join(current_path, 'pro_train_ekphrasis.csv')
processed_dev_ekphrasis_path = os.path.join(current_path, 'pro_dev_ekphrasis.csv')
processed_test_ekphrasis_path = os.path.join(current_path, 'pro_test_ekphrasis.csv')

In [6]:
train = pd.read_csv(train_ekphrasis_path)
train.info()
display(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14066 entries, 0 to 14065
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_id       14066 non-null  int64 
 1   post_text     14066 non-null  object
 2   user_id       14066 non-null  int64 
 3   image_id(s)   14066 non-null  object
 4   username      14066 non-null  object
 5   timestamp     14066 non-null  object
 6   label         14066 non-null  object
 7   cleaned_text  14066 non-null  object
 8   enc_label     14066 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 989.1+ KB


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263528431088721921,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,204337678,sandy_fake_57,ImObiWanKenobi,Wed Oct 31 06:30:48 +0000 2012,fake,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,0
1,263493828349145088,Dicen que despues de la tormenta llega la calm...,619474112,sandy_real_114,1UniikeDiiva,Wed Oct 31 04:13:18 +0000 2012,real,Dicen que despues de la tormenta llega la calm...,1
2,263060464571912192,a SHARK swam up to brigantine.. \n#hurricanesa...,19117875,sandy_fake_26,aimee_sh,Mon Oct 29 23:31:16 +0000 2012,fake,a SHARK swam up to brigantine.. \n#hurricanesa...,0
3,262961670551400448,Tomb of the Unknown Soldier during Sandy. #muc...,67930605,sandy_fake_63,thuntley11,Mon Oct 29 16:58:42 +0000 2012,fake,Tomb of the Unknown Soldier during Sandy. #muc...,0
4,263086042549194752,Manhattan holy shit #hurricanesandy http://t.c...,263461477,sandy_real_105,CatieBoel,Tue Oct 30 01:12:55 +0000 2012,real,Manhattan holy shit #hurricanesandy http://t.c...,1


In [7]:
dev = pd.read_csv(dev_ekphrasis_path)
dev.info()
display(dev.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_id       1563 non-null   int64 
 1   post_text     1563 non-null   object
 2   user_id       1563 non-null   int64 
 3   image_id(s)   1563 non-null   object
 4   username      1563 non-null   object
 5   timestamp     1563 non-null   object
 6   label         1563 non-null   object
 7   cleaned_text  1563 non-null   object
 8   enc_label     1563 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 110.0+ KB


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263021327630340096,#sandy http://t.co/Ub6QRUcl,271599347,sandy_fake_39,OscarKennish,Mon Oct 29 20:55:45 +0000 2012,fake,#sandy http://t.co/Ub6QRUcl,0
1,263062792662286337,Hayyyy nanita!!!! EN VIVO #SANDY / ESTATUA DE ...,174872551,sandy_fake_21,Betodiaz79,Mon Oct 29 23:40:31 +0000 2012,fake,Hayyyy nanita!!!! EN VIVO #SANDY / ESTATUA DE ...,0
2,264289224314994688,So shines a good deed in a naughty world #Sand...,10836092,sandy_real_71,joehas,Fri Nov 02 08:53:56 +0000 2012,real,So shines a good deed in a naughty world #Sand...,1
3,263015954211016705,Tomb of the Unknown Soldier continues to stay ...,562635567,sandy_fake_16,bbarnessoccer95,Mon Oct 29 20:34:24 +0000 2012,fake,Tomb of the Unknown Soldier continues to stay ...,0
4,532632521523998720,Young boy demonstrates heroic virtue.--SYRIAN ...,795540624,syrianboy_1,Deborah41006526,Wed Nov 12 20:34:23 +0000 2014,fake,Young boy demonstrates heroic virtue.--SYRIAN ...,0


In [8]:
test = pd.read_csv(test_ekphrasis_path)
test.info()
display(test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   post_id       2177 non-null   int64  
 1   post_text     2177 non-null   object 
 2   user_id       2177 non-null   float64
 3   username      2177 non-null   object 
 4   image_id      2177 non-null   object 
 5   timestamp     2177 non-null   object 
 6   label         2177 non-null   object 
 7   cleaned_text  2177 non-null   object 
 8   enc_label     2177 non-null   int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 153.2+ KB


,post_id,post_text,user_id,username,image_id,timestamp,label,cleaned_text,enc_label
0,651118294447951872,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:34:33 +0000 2015,fake,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",0
1,651115824065830912,"#АнтиТеррор ""#АрабскаяКровь"" #РФ в #Сирии прот...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:24:44 +0000 2015,fake,"#АнтиТеррор ""#АрабскаяКровь"" #РФ в #Сирии прот...",0
2,651095856662360065,"Ну чё, сегодня надавали трендюлей игиловцам по...",2.712310e+09,NataYaraya,airstrikes_1,Mon Oct 05 18:05:23 +0000 2015,fake,"Ну чё, сегодня надавали трендюлей игиловцам по...",0
3,651086828234104832,Действия России в Сирии безукоризненны. Видео...,3.690572e+07,Alltecz,airstrikes_1,Mon Oct 05 17:29:31 +0000 2015,fake,Действия России в Сирии безукоризненны. Видео...,0
4,651034616007106560,5-10-2015\nThe airstrike against an ISIS ammun...,1.070959e+09,msojormsojor,airstrikes_1,Mon Oct 05 14:02:02 +0000 2015,fake,5-10-2015\nThe airstrike against an ISIS ammun...,0


In [9]:
# %pip install ekphrasis

In [10]:
# %pip install tweet-preprocessor

In [11]:
import preprocessor as p

In [12]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'user'],
    fix_html=True,  # fix HTML tokens
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons],
    mode='fast'
)

/Users/wangkunpeng/miniconda3/envs/mlp/lib/python3.8/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))
/Users/wangkunpeng/miniconda3/envs/mlp/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [13]:
translator = Translator()

In [14]:
def preprocessing(text):
  p.set_options(p.OPT.RESERVED) #Delete RT
  text = p.clean(text)

  text = re.sub(r'\n', ' ', text)
  text = re.sub(r'\\n', ' ', text)
  text = re.sub(r'(&amp;|&gt;|&lt;)', " ", text) #Delete some special words
  text = " ".join(text_processor.pre_process_doc(text))
  text = re.sub(r"\s+", " ", text)
  
  return text  

In [15]:
chinese_text = "我是精神美国人，只抽电子烟，从不看战狼，不用华为只用苹果手机"
japanese_text="私はスピリチュアルなアメリカ人です。電子タバコしか吸わないし、Wolf Warriors も見ません。Huawei も使用せず、Apple の電話しか使用しません"
english_text = translator.translate(japanese_text, src='auto', dest='en').text
print(english_text)

I am a spiritual American. I only smoke e-cigarettes and I don't watch Wolf Warriors. I don't even use Huawei, I only use Apple phones


In [24]:
import tqdm

# train['cleaned_text'] = train['post_text']
# dev['cleaned_text'] = dev['post_text']
# test['cleaned_text'] = test['post_text']

for i in tqdm.tqdm(range(len(train['post_text']))):
  sent = train['cleaned_text'][i]
  sent = preprocessing(sent)
  train['cleaned_text'][i] = translator.translate(sent, src='auto', dest='en').text

  0%|                                                 | 0/14066 [00:00<?, ?it/s]/var/folders/r6/6qjzb__x4_n3hg_dp7c2hqgh0000gn/T/ipykernel_1679/2655611154.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['cleaned_text'][i] = translator.translate(sent, src='auto', dest='en').text
 25%|█████████▌                            | 3553/14066 [10:26<30:53,  5.67it/s]


ConnectTimeout: _ssl.c:1114: The handshake operation timed out

In [25]:
for i in tqdm.tqdm(range(len(dev['post_text']))):
  sent = dev['cleaned_text'][i]
  sent = preprocessing(sent)
  dev['cleaned_text'][i] = translator.translate(sent, src='auto', dest='en').text

  0%|                                                  | 0/1563 [00:00<?, ?it/s]/var/folders/r6/6qjzb__x4_n3hg_dp7c2hqgh0000gn/T/ipykernel_1679/2922814326.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['cleaned_text'][i] = translator.translate(sent, src='auto', dest='en').text
100%|███████████████████████████████████████| 1563/1563 [04:03<00:00,  6.42it/s]


In [ ]:
for i in tqdm.tqdm(range(len(test['post_text']))):
  sent = test['cleaned_text'][i]
  sent = preprocessing(sent)
  test['cleaned_text'][i] = translator.translate(sent, src='auto', dest='en').text

In [18]:
train['enc_label'] = train['label']
train['enc_label'] = train['enc_label'].apply(lambda x: 1 if x=='fake' else 0)
dev['enc_label'] = dev['label']
dev['enc_label'] = dev['enc_label'].apply(lambda x: 1 if x=='fake' else 0)
test['enc_label'] = test['label']
test['enc_label'] = test['enc_label'].apply(lambda x: 1 if x=='fake' else 0)

In [19]:
display(train.head()), display(dev.head()), display(test.head())

,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263528431088721921,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,204337678,sandy_fake_57,ImObiWanKenobi,Wed Oct 31 06:30:48 +0000 2012,fake,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,1
1,263493828349145088,Dicen que despues de la tormenta llega la calm...,619474112,sandy_real_114,1UniikeDiiva,Wed Oct 31 04:13:18 +0000 2012,real,They say that after the storm comes the calm! ...,0
2,263060464571912192,a SHARK swam up to brigantine.. \n#hurricanesa...,19117875,sandy_fake_26,aimee_sh,Mon Oct 29 23:31:16 +0000 2012,fake,a SHARK swam up to brigantine.. \n#hurricanesa...,1
3,262961670551400448,Tomb of the Unknown Soldier during Sandy. #muc...,67930605,sandy_fake_63,thuntley11,Mon Oct 29 16:58:42 +0000 2012,fake,Tomb of the Unknown Soldier during Sandy. #muc...,1
4,263086042549194752,Manhattan holy shit #hurricanesandy http://t.c...,263461477,sandy_real_105,CatieBoel,Tue Oct 30 01:12:55 +0000 2012,real,Manhattan holy shit #hurricanesandy http://t.c...,0


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263021327630340096,#sandy http://t.co/Ub6QRUcl,271599347,sandy_fake_39,OscarKennish,Mon Oct 29 20:55:45 +0000 2012,fake,#sandy http://t.co/Ub6QRUcl,1
1,263062792662286337,Hayyyy nanita!!!! EN VIVO #SANDY / ESTATUA DE ...,174872551,sandy_fake_21,Betodiaz79,Mon Oct 29 23:40:31 +0000 2012,fake,Hayyyy nanita!!!! LIVE #SANDY / STATUE OF LIBE...,1
2,264289224314994688,So shines a good deed in a naughty world #Sand...,10836092,sandy_real_71,joehas,Fri Nov 02 08:53:56 +0000 2012,real,So shines a good deed in a naughty world #Sand...,0
3,263015954211016705,Tomb of the Unknown Soldier continues to stay ...,562635567,sandy_fake_16,bbarnessoccer95,Mon Oct 29 20:34:24 +0000 2012,fake,Tomb of the Unknown Soldier continues to stay ...,1
4,532632521523998720,Young boy demonstrates heroic virtue.--SYRIAN ...,795540624,syrianboy_1,Deborah41006526,Wed Nov 12 20:34:23 +0000 2014,fake,Young boy demonstrates heroic virtue.--SYRIAN ...,1


,post_id,post_text,user_id,username,image_id,timestamp,label,cleaned_text,enc_label
0,651118294447951872,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:34:33 +0000 2015,fake,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",1
1,651115824065830912,"#АнтиТеррор ""#АрабскаяКровь"" #РФ в #Сирии прот...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:24:44 +0000 2015,fake,"#Anti-Terror ""#ArabBlood"" #RF in #Syria agains...",1
2,651095856662360065,"Ну чё, сегодня надавали трендюлей игиловцам по...",2.712310e+09,NataYaraya,airstrikes_1,Mon Oct 05 18:05:23 +0000 2015,fake,"Well, today they gave tryndula to the Igilovit...",1
3,651086828234104832,Действия России в Сирии безукоризненны. Видео...,3.690572e+07,Alltecz,airstrikes_1,Mon Oct 05 17:29:31 +0000 2015,fake,Russia's actions in Syria are irreproachable. ...,1
4,651034616007106560,5-10-2015\nThe airstrike against an ISIS ammun...,1.070959e+09,msojormsojor,airstrikes_1,Mon Oct 05 14:02:02 +0000 2015,fake,5-10-2015\nThe airstrike against an ISIS ammun...,1


(None, None, None)

In [23]:
train.to_csv(processed_train_ekphrasis_path,index=False)
dev.to_csv(processed_dev_ekphrasis_path,index=False)
test.to_csv(processed_test_ekphrasis_path,index=False)

In [26]:
train = pd.read_csv(processed_train_ekphrasis_path)
dev = pd.read_csv(processed_dev_ekphrasis_path)
test = pd.read_csv(processed_test_ekphrasis_path)

display(train.head()), display(dev.head()), display(test.head())

,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263528431088721921,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,204337678,sandy_fake_57,ImObiWanKenobi,Wed Oct 31 06:30:48 +0000 2012,fake,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,1
1,263493828349145088,Dicen que despues de la tormenta llega la calm...,619474112,sandy_real_114,1UniikeDiiva,Wed Oct 31 04:13:18 +0000 2012,real,They say that after the storm comes the calm! ...,0
2,263060464571912192,a SHARK swam up to brigantine.. \n#hurricanesa...,19117875,sandy_fake_26,aimee_sh,Mon Oct 29 23:31:16 +0000 2012,fake,a SHARK swam up to brigantine.. \n#hurricanesa...,1
3,262961670551400448,Tomb of the Unknown Soldier during Sandy. #muc...,67930605,sandy_fake_63,thuntley11,Mon Oct 29 16:58:42 +0000 2012,fake,Tomb of the Unknown Soldier during Sandy. #muc...,1
4,263086042549194752,Manhattan holy shit #hurricanesandy http://t.c...,263461477,sandy_real_105,CatieBoel,Tue Oct 30 01:12:55 +0000 2012,real,Manhattan holy shit #hurricanesandy http://t.c...,0


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263021327630340096,#sandy http://t.co/Ub6QRUcl,271599347,sandy_fake_39,OscarKennish,Mon Oct 29 20:55:45 +0000 2012,fake,#sandy http://t.co/Ub6QRUcl,1
1,263062792662286337,Hayyyy nanita!!!! EN VIVO #SANDY / ESTATUA DE ...,174872551,sandy_fake_21,Betodiaz79,Mon Oct 29 23:40:31 +0000 2012,fake,Hayyyy nanita!!!! LIVE #SANDY / STATUE OF LIBE...,1
2,264289224314994688,So shines a good deed in a naughty world #Sand...,10836092,sandy_real_71,joehas,Fri Nov 02 08:53:56 +0000 2012,real,So shines a good deed in a naughty world #Sand...,0
3,263015954211016705,Tomb of the Unknown Soldier continues to stay ...,562635567,sandy_fake_16,bbarnessoccer95,Mon Oct 29 20:34:24 +0000 2012,fake,Tomb of the Unknown Soldier continues to stay ...,1
4,532632521523998720,Young boy demonstrates heroic virtue.--SYRIAN ...,795540624,syrianboy_1,Deborah41006526,Wed Nov 12 20:34:23 +0000 2014,fake,Young boy demonstrates heroic virtue.--SYRIAN ...,1


,post_id,post_text,user_id,username,image_id,timestamp,label,cleaned_text,enc_label
0,651118294447951872,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:34:33 +0000 2015,fake,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",1
1,651115824065830912,"#АнтиТеррор ""#АрабскаяКровь"" #РФ в #Сирии прот...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:24:44 +0000 2015,fake,"#Anti-Terror ""#ArabBlood"" #RF in #Syria agains...",1
2,651095856662360065,"Ну чё, сегодня надавали трендюлей игиловцам по...",2.712310e+09,NataYaraya,airstrikes_1,Mon Oct 05 18:05:23 +0000 2015,fake,"Well, today they gave tryndula to the Igilovit...",1
3,651086828234104832,Действия России в Сирии безукоризненны. Видео...,3.690572e+07,Alltecz,airstrikes_1,Mon Oct 05 17:29:31 +0000 2015,fake,Russia's actions in Syria are irreproachable. ...,1
4,651034616007106560,5-10-2015\nThe airstrike against an ISIS ammun...,1.070959e+09,msojormsojor,airstrikes_1,Mon Oct 05 14:02:02 +0000 2015,fake,5-10-2015\nThe airstrike against an ISIS ammun...,1


(None, None, None)

In [29]:
import tqdm

# train['cleaned_text'] = train['post_text']
# dev['cleaned_text'] = dev['post_text']
# test['cleaned_text'] = test['post_text']

for i in tqdm.tqdm(range(len(train['post_text']))):
  sent = train['cleaned_text'][i]
  sent = preprocessing(sent)
  train['cleaned_text'][i] = sent

for i in tqdm.tqdm(range(len(dev['post_text']))):
  sent = dev['cleaned_text'][i]
  sent = preprocessing(sent)
  dev['cleaned_text'][i] = sent

for i in tqdm.tqdm(range(len(test['post_text']))):
  sent = test['cleaned_text'][i]
  sent = preprocessing(sent)
  test['cleaned_text'][i] = sent

  0%|                                                 | 0/14066 [00:00<?, ?it/s]/var/folders/r6/6qjzb__x4_n3hg_dp7c2hqgh0000gn/T/ipykernel_1679/1263290442.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['cleaned_text'][i] = sent
  0%|                                                  | 0/1563 [00:00<?, ?it/s]/var/folders/r6/6qjzb__x4_n3hg_dp7c2hqgh0000gn/T/ipykernel_1679/1263290442.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['cleaned_text'][i] = sent
  0%|                                                  | 0/2177 [00:00<?, ?it/s]/var/folders/r6/6qjzb__x4_n3hg_dp7c2hqgh0000gn/T/ipykernel_1679/1263

In [30]:
display(train.head()), display(dev.head()), display(test.head())

,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263528431088721921,Hurricane SANDY-CAT . 🐱 Meow ~ http://t.co/dj7...,204337678,sandy_fake_57,ImObiWanKenobi,Wed Oct 31 06:30:48 +0000 2012,fake,Hurricane SANDY - CAT . 🐱 Meow ~ <url>,1
1,263493828349145088,Dicen que despues de la tormenta llega la calm...,619474112,sandy_real_114,1UniikeDiiva,Wed Oct 31 04:13:18 +0000 2012,real,They say that after the storm comes the calm !...,0
2,263060464571912192,a SHARK swam up to brigantine.. \n#hurricanesa...,19117875,sandy_fake_26,aimee_sh,Mon Oct 29 23:31:16 +0000 2012,fake,a SHARK swam up to brigantine . . #hurricanesa...,1
3,262961670551400448,Tomb of the Unknown Soldier during Sandy. #muc...,67930605,sandy_fake_63,thuntley11,Mon Oct 29 16:58:42 +0000 2012,fake,Tomb of the Unknown Soldier during Sandy . #mu...,1
4,263086042549194752,Manhattan holy shit #hurricanesandy http://t.c...,263461477,sandy_real_105,CatieBoel,Tue Oct 30 01:12:55 +0000 2012,real,Manhattan holy shit #hurricanesandy <url>,0


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263021327630340096,#sandy http://t.co/Ub6QRUcl,271599347,sandy_fake_39,OscarKennish,Mon Oct 29 20:55:45 +0000 2012,fake,#sandy <url>,1
1,263062792662286337,Hayyyy nanita!!!! EN VIVO #SANDY / ESTATUA DE ...,174872551,sandy_fake_21,Betodiaz79,Mon Oct 29 23:40:31 +0000 2012,fake,Hayyyy nanita ! ! ! ! LIVE #SANDY / STATUE OF ...,1
2,264289224314994688,So shines a good deed in a naughty world #Sand...,10836092,sandy_real_71,joehas,Fri Nov 02 08:53:56 +0000 2012,real,So shines a good deed in a naughty world #Sand...,0
3,263015954211016705,Tomb of the Unknown Soldier continues to stay ...,562635567,sandy_fake_16,bbarnessoccer95,Mon Oct 29 20:34:24 +0000 2012,fake,Tomb of the Unknown Soldier continues to stay ...,1
4,532632521523998720,Young boy demonstrates heroic virtue.--SYRIAN ...,795540624,syrianboy_1,Deborah41006526,Wed Nov 12 20:34:23 +0000 2014,fake,Young boy demonstrates heroic virtue . - - SYR...,1


,post_id,post_text,user_id,username,image_id,timestamp,label,cleaned_text,enc_label
0,651118294447951872,"#AntiTerror ""#ArabianBlood"" #RussianFederation...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:34:33 +0000 2015,fake,"#AntiTerror "" #ArabianBlood "" #RussianFederati...",1
1,651115824065830912,"#АнтиТеррор ""#АрабскаяКровь"" #РФ в #Сирии прот...",3.834097e+08,AlexArtAndros,airstrikes_1,Mon Oct 05 19:24:44 +0000 2015,fake,"#Anti-Terror "" #ArabBlood "" #RF in #Syria agai...",1
2,651095856662360065,"Ну чё, сегодня надавали трендюлей игиловцам по...",2.712310e+09,NataYaraya,airstrikes_1,Mon Oct 05 18:05:23 +0000 2015,fake,"Well , today they gave tryndula to the Igilovi...",1
3,651086828234104832,Действия России в Сирии безукоризненны. Видео...,3.690572e+07,Alltecz,airstrikes_1,Mon Oct 05 17:29:31 +0000 2015,fake,Russia ' s actions in Syria are irreproachable...,1
4,651034616007106560,5-10-2015\nThe airstrike against an ISIS ammun...,1.070959e+09,msojormsojor,airstrikes_1,Mon Oct 05 14:02:02 +0000 2015,fake,5-10-2015 The airstrike against an ISIS ammuni...,1


(None, None, None)

In [31]:
train.to_csv(processed_train_ekphrasis_path,index=False)
dev.to_csv(processed_dev_ekphrasis_path,index=False)
test.to_csv(processed_test_ekphrasis_path,index=False)